# Regresión logística

- La regresión logística la usamos cuando la variable resultado no es continua, es discreta -> problemas de clasificación 
    - (vivo o muerto en el Titanic) - clase, hombre o mujer, niño o adulto, si viaja sólo o acompañado
    - un cliente va a comprar o no - sexo, edad...
    - que equipo de fútbol va a ganar una copa - datos de los jugadores, estadios, fechas, clima..
    



In [1]:
import pandas as pd
import numpy as np 


In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/afoone/caipc-mar-2023/master/data/titanic/train.csv')

In [3]:
test = pd.read_csv('https://raw.githubusercontent.com/afoone/caipc-mar-2023/master/data/titanic/test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train.shape

(891, 12)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
#train['Ticket'].unique()

In [8]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']

train = train.drop(drop_cols, axis=1)

In [10]:
test = test.drop(drop_cols, axis=1)

In [11]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [12]:
dummies_embarked = pd.get_dummies(train['Embarked'], prefix='port')
dummies_gender = pd.get_dummies(train['Sex'], prefix='gender')
dummies_class = pd.get_dummies(train['Pclass'], prefix='class')

In [13]:
dummies_class

,class_1,class_2,class_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1
...,...,...,...
886,0,1,0
887,1,0,0
888,0,0,1
889,1,0,0


In [14]:
train = train.join(dummies_class).join(dummies_embarked).join(dummies_gender)

In [15]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,class_1,class_2,class_3,port_C,port_Q,port_S,gender_female,gender_male
0,0,3,male,22.0,1,0,7.2500,S,0,0,1,0,0,1,0,1
1,1,1,female,38.0,1,0,71.2833,C,1,0,0,1,0,0,1,0
2,1,3,female,26.0,0,0,7.9250,S,0,0,1,0,0,1,1,0
3,1,1,female,35.0,1,0,53.1000,S,1,0,0,0,0,1,1,0
4,0,3,male,35.0,0,0,8.0500,S,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0,1,0,0,0,1,0,1
887,1,1,female,19.0,0,0,30.0000,S,1,0,0,0,0,1,1,0
888,0,3,female,NaN,1,2,23.4500,S,0,0,1,0,0,1,1,0
889,1,1,male,26.0,0,0,30.0000,C,1,0,0,1,0,0,0,1


In [16]:
dummies_embarked = pd.get_dummies(test['Embarked'], prefix='port')
dummies_gender = pd.get_dummies(test['Sex'], prefix='gender')
dummies_class = pd.get_dummies(test['Pclass'], prefix='class')
test = test.join(dummies_class).join(dummies_embarked).join(dummies_gender)

In [17]:
test.head() # esto es el submit a kaggle; no vamos a continuar con esto

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,class_1,class_2,class_3,port_C,port_Q,port_S,gender_female,gender_male
0,3,male,34.5,0,0,7.8292,Q,0,0,1,0,1,0,0,1
1,3,female,47.0,1,0,7.0000,S,0,0,1,0,0,1,1,0
2,2,male,62.0,0,0,9.6875,Q,0,1,0,0,1,0,0,1
3,3,male,27.0,0,0,8.6625,S,0,0,1,0,0,1,0,1
4,3,female,22.0,1,1,12.2875,S,0,0,1,0,0,1,1,0


In [18]:
df = train.copy()

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       891 non-null    int64  
 1   Pclass         891 non-null    int64  
 2   Sex            891 non-null    object 
 3   Age            714 non-null    float64
 4   SibSp          891 non-null    int64  
 5   Parch          891 non-null    int64  
 6   Fare           891 non-null    float64
 7   Embarked       889 non-null    object 
 8   class_1        891 non-null    uint8  
 9   class_2        891 non-null    uint8  
 10  class_3        891 non-null    uint8  
 11  port_C         891 non-null    uint8  
 12  port_Q         891 non-null    uint8  
 13  port_S         891 non-null    uint8  
 14  gender_female  891 non-null    uint8  
 15  gender_male    891 non-null    uint8  
dtypes: float64(2), int64(4), object(2), uint8(8)
memory usage: 62.8+ KB


In [20]:
df_age = df.dropna()

In [21]:
df_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Survived       712 non-null    int64  
 1   Pclass         712 non-null    int64  
 2   Sex            712 non-null    object 
 3   Age            712 non-null    float64
 4   SibSp          712 non-null    int64  
 5   Parch          712 non-null    int64  
 6   Fare           712 non-null    float64
 7   Embarked       712 non-null    object 
 8   class_1        712 non-null    uint8  
 9   class_2        712 non-null    uint8  
 10  class_3        712 non-null    uint8  
 11  port_C         712 non-null    uint8  
 12  port_Q         712 non-null    uint8  
 13  port_S         712 non-null    uint8  
 14  gender_female  712 non-null    uint8  
 15  gender_male    712 non-null    uint8  
dtypes: float64(2), int64(4), object(2), uint8(8)
memory usage: 55.6+ KB


In [22]:
train.columns


Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'class_1', 'class_2', 'class_3', 'port_C', 'port_Q',
       'port_S', 'gender_female', 'gender_male'],
      dtype='object')

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_age[[ 'Age', 'SibSp', 'Parch', 'Fare',
        'class_1', 'class_2', 'class_3', 'port_C', 'port_Q',
       'port_S', 'gender_female', 'gender_male']]
    , df_age['Survived'], train_size=0.8, shuffle=True )

In [24]:
X_train

,Age,SibSp,Parch,Fare,class_1,class_2,class_3,port_C,port_Q,port_S,gender_female,gender_male
529,23.0,2,1,11.5000,0,1,0,0,0,1,0,1
313,28.0,0,0,7.8958,0,0,1,0,0,1,0,1
4,35.0,0,0,8.0500,0,0,1,0,0,1,0,1
86,16.0,1,3,34.3750,0,0,1,0,0,1,0,1
317,54.0,0,0,14.0000,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
455,29.0,0,0,7.8958,0,0,1,1,0,0,0,1
360,40.0,1,4,27.9000,0,0,1,0,0,1,0,1
387,36.0,0,0,13.0000,0,1,0,0,0,1,1,0
448,5.0,2,1,19.2583,0,0,1,1,0,0,1,0


In [25]:
X_test


,Age,SibSp,Parch,Fare,class_1,class_2,class_3,port_C,port_Q,port_S,gender_female,gender_male
600,24.0,2,1,27.000,0,1,0,0,0,1,1,0
615,24.0,1,2,65.000,0,1,0,0,0,1,1,0
363,35.0,0,0,7.050,0,0,1,0,0,1,0,1
814,30.5,0,0,8.050,0,0,1,0,0,1,0,1
242,29.0,0,0,10.500,0,1,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
683,14.0,5,2,46.900,0,0,1,0,0,1,0,1
15,55.0,0,0,16.000,0,1,0,0,0,1,1,0
210,24.0,0,0,7.050,0,0,1,0,0,1,0,1
244,30.0,0,0,7.225,0,0,1,1,0,0,0,1


In [26]:
# seleccion de variables
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression



In [27]:
model = LogisticRegression(solver='liblinear')

In [28]:
model

LogisticRegression(solver='liblinear')

In [29]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 569 entries, 529 to 474
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Age            569 non-null    float64
 1   SibSp          569 non-null    int64  
 2   Parch          569 non-null    int64  
 3   Fare           569 non-null    float64
 4   class_1        569 non-null    uint8  
 5   class_2        569 non-null    uint8  
 6   class_3        569 non-null    uint8  
 7   port_C         569 non-null    uint8  
 8   port_Q         569 non-null    uint8  
 9   port_S         569 non-null    uint8  
 10  gender_female  569 non-null    uint8  
 11  gender_male    569 non-null    uint8  
dtypes: float64(2), int64(2), uint8(8)
memory usage: 26.7 KB


In [30]:
y_train

529    0
313    0
4      0
86     0
317    0
      ..
455    1
360    0
387    1
448    1
474    0
Name: Survived, Length: 569, dtype: int64

In [40]:
seleccion = RFE(model, n_features_to_select=4, step=1).fit(X_train, y_train)

In [41]:
seleccion.support_

array([False, False, False, False,  True, False,  True, False, False,
       False,  True,  True])

In [42]:
X_train.columns

Index(['Age', 'SibSp', 'Parch', 'Fare', 'class_1', 'class_2', 'class_3',
       'port_C', 'port_Q', 'port_S', 'gender_female', 'gender_male'],
      dtype='object')

In [43]:
seleccion.ranking_


array([7, 4, 8, 9, 1, 6, 1, 2, 3, 5, 1, 1])

In [44]:
list(
    zip(
        X_train.columns,
        seleccion.ranking_
    )
)

[('Age', 7),
 ('SibSp', 4),
 ('Parch', 8),
 ('Fare', 9),
 ('class_1', 1),
 ('class_2', 6),
 ('class_3', 1),
 ('port_C', 2),
 ('port_Q', 3),
 ('port_S', 5),
 ('gender_female', 1),
 ('gender_male', 1)]

In [45]:
model.fit(
    X_train[['class_1', 'class_3', 'gender_female', 'gender_male']],
    y_train
)

LogisticRegression(solver='liblinear')

In [47]:
model.score(
        X_train[['class_1', 'class_3', 'gender_female', 'gender_male']],
    y_train
)

0.7609841827768014

In [48]:
predicciones = model.predict(
    X_test[['class_1', 'class_3', 'gender_female', 'gender_male']]
)

In [49]:
predicciones

array([1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1])

In [50]:
y_test


600    1
615    1
363    0
814    0
242    0
      ..
683    0
15     1
210    0
244    0
190    1
Name: Survived, Length: 143, dtype: int64

In [52]:
validation = y_test.copy()

In [54]:
validation = pd.DataFrame()

In [55]:
validation['y'] = y_test

In [56]:
validation['predicted'] = predicciones

In [60]:
(validation['predicted'] == validation['y']).sum()/ len(validation)

0.8531468531468531

In [63]:
model.predict_proba(
    X_test[['class_1', 'class_3', 'gender_female', 'gender_male']]
)

array([[0.23987135, 0.76012865],
       [0.23987135, 0.76012865],
       [0.88775438, 0.11224562],
       [0.88775438, 0.11224562],
       [0.75269424, 0.24730576],
       [0.45055885, 0.54944115],
       [0.88775438, 0.11224562],
       [0.45055885, 0.54944115],
       [0.88775438, 0.11224562],
       [0.45055885, 0.54944115],
       [0.88775438, 0.11224562],
       [0.45055885, 0.54944115],
       [0.45055885, 0.54944115],
       [0.45055885, 0.54944115],
       [0.45055885, 0.54944115],
       [0.88775438, 0.11224562],
       [0.45055885, 0.54944115],
       [0.88775438, 0.11224562],
       [0.88775438, 0.11224562],
       [0.88775438, 0.11224562],
       [0.75269424, 0.24730576],
       [0.88775438, 0.11224562],
       [0.75269424, 0.24730576],
       [0.75269424, 0.24730576],
       [0.88775438, 0.11224562],
       [0.23987135, 0.76012865],
       [0.88775438, 0.11224562],
       [0.88775438, 0.11224562],
       [0.75269424, 0.24730576],
       [0.75269424, 0.24730576],
       [0.

# Validación cruzada

In [64]:
from sklearn.model_selection import cross_val_score

In [65]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,class_1,class_2,class_3,port_C,port_Q,port_S,gender_female,gender_male
0,0,3,male,22.0,1,0,7.2500,S,0,0,1,0,0,1,0,1
1,1,1,female,38.0,1,0,71.2833,C,1,0,0,1,0,0,1,0
2,1,3,female,26.0,0,0,7.9250,S,0,0,1,0,0,1,1,0
3,1,1,female,35.0,1,0,53.1000,S,1,0,0,0,0,1,1,0
4,0,3,male,35.0,0,0,8.0500,S,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0,1,0,0,0,1,0,1
887,1,1,female,19.0,0,0,30.0000,S,1,0,0,0,0,1,1,0
888,0,3,female,NaN,1,2,23.4500,S,0,0,1,0,0,1,1,0
889,1,1,male,26.0,0,0,30.0000,C,1,0,0,1,0,0,0,1


In [66]:
result = cross_val_score(
    LogisticRegression(solver='liblinear'),
    df[['class_1', 'class_3', 'gender_female', 'gender_male']],
    df['Survived'],
    scoring='accuracy',
    cv=5
)

In [68]:
result

array([0.80446927, 0.80337079, 0.78651685, 0.75280899, 0.78651685])

In [69]:
result.mean()

0.7867365513778168

In [70]:
result.var()

0.0003484646519976675